In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

## This notebook outlines the high level industry data formatting for the Comprehensive Plans with place level primary geographies. There is another document for the information that is pulled for all relevant geographies.

The Woods & Poole data that is used more at the county level is only used at the place level for total employment projections. This is for consistency. This is a land use model ultimately, so JobsEQ employment is selected to model place level employment for place of work. This data is derived from the QCEW and other inputs so is selected as a better option - both are modeled so we use the one modeled directly from UI claims rather than land use at this granular of a geography.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ Select the "Industry" dataset, annual  
+ Select the following dimensions: Region, Industry, Year/Quarter, then select "Employment" as the value  
+ Under "Region" select "GNRC Region" and sub "Census Place"  
+ For "Year/Quarter" select as many as possible  
+ Under "Industry" expand "Total-All Industries" and select 2 digit NAICS codes

This may exceed the drafting size, in which case you'll have to queu for download. Once it is completed, open and delete the top row that says "Industry Data", and delete the third row that repeats "Employment". Rename the FIPS column to GEO_ID, "Time" to "Year", and "Region" to "NAME". Use text to columns to change the GEO_ID column to text. Save in the "Data Downloads" folder as "JobsEQIndustryEmployment_Place.csv"

In [291]:
data = pd.read_csv('../../Data Downloads/JobsEQIndustryEmployment_Placetest.csv')

In [292]:
data.head()

,NAME,Year,Total - All Industries,"Agriculture, Forestry, Fishing and Hunting (11)","Mining, Quarrying, and Oil and Gas Extraction (21)",Utilities (22),Construction (23),Manufacturing (31),Wholesale Trade (42),Retail Trade (44),Transportation and Warehousing (48),Information (51),Finance and Insurance (52),Real Estate and Rental and Leasing (53),"Professional, Scientific, and Technical Services (54)",Management of Companies and Enterprises (55),Administrative and Support and Waste Management and Remediation Services (56),Educational Services (61),Health Care and Social Assistance (62),"Arts, Entertainment, and Recreation (71)",Accommodation and Food Services (72),Other Services (except Public Administration) (81),Public Administration (92),Unclassified (99)
0,GNRC Region,2001.0,833913.0,9154.0,544.0,4967.0,52041.0,105210.0,36461.0,92291.0,33036.0,24400.0,37052.0,15202.0,37355.0,5342.0,61260.0,53487.0,85812.0,12839.0,67647.0,41599.0,57873.0,342.0
1,GNRC Region,2002.0,828380.0,9122.0,484.0,4936.0,50012.0,99314.0,35301.0,89659.0,31977.0,23209.0,35931.0,14402.0,38070.0,6300.0,62009.0,55647.0,88519.0,13160.0,68036.0,42987.0,58987.0,316.0
2,GNRC Region,2003.0,844281.0,8702.0,467.0,4890.0,52876.0,97117.0,37749.0,92188.0,33758.0,21799.0,36908.0,15300.0,41033.0,6642.0,53743.0,58466.0,93038.0,13908.0,70520.0,43828.0,60833.0,516.0
3,GNRC Region,2004.0,865173.0,8652.0,439.0,4729.0,53160.0,97956.0,37553.0,96070.0,34691.0,21040.0,36820.0,14964.0,41510.0,7225.0,61188.0,62132.0,95216.0,14213.0,72177.0,43886.0,61072.0,478.0
4,GNRC Region,2005.0,886623.0,8463.0,550.0,4723.0,55530.0,97997.0,38552.0,98949.0,35240.0,21558.0,37089.0,15330.0,43077.0,7202.0,64118.0,64691.0,97755.0,14648.0,75825.0,43523.0,61470.0,336.0


In [293]:
data = data.set_index(['NAME'])#, 'GEO_ID'])
cols = data.columns
data[cols] = data[cols].astype(float)

In [271]:
data.head()

,Year,Total - All Industries,"Agriculture, Forestry, Fishing and Hunting (11)","Mining, Quarrying, and Oil and Gas Extraction (21)",Utilities (22),Construction (23),Manufacturing (31),Wholesale Trade (42),Retail Trade (44),Transportation and Warehousing (48),Information (51),Finance and Insurance (52),Real Estate and Rental and Leasing (53),"Professional, Scientific, and Technical Services (54)",Management of Companies and Enterprises (55),Administrative and Support and Waste Management and Remediation Services (56),Educational Services (61),Health Care and Social Assistance (62),"Arts, Entertainment, and Recreation (71)",Accommodation and Food Services (72),Other Services (except Public Administration) (81),Public Administration (92),Unclassified (99)
NAME,,,,,,,,,,,,,,,,,,,,,,,
GNRC Region,2001.0,833913.0,9154.0,544.0,4967.0,52041.0,105210.0,36461.0,92291.0,33036.0,24400.0,37052.0,15202.0,37355.0,5342.0,61260.0,53487.0,85812.0,12839.0,67647.0,41599.0,57873.0,342.0
GNRC Region,2002.0,828380.0,9122.0,484.0,4936.0,50012.0,99314.0,35301.0,89659.0,31977.0,23209.0,35931.0,14402.0,38070.0,6300.0,62009.0,55647.0,88519.0,13160.0,68036.0,42987.0,58987.0,316.0
GNRC Region,2003.0,844281.0,8702.0,467.0,4890.0,52876.0,97117.0,37749.0,92188.0,33758.0,21799.0,36908.0,15300.0,41033.0,6642.0,53743.0,58466.0,93038.0,13908.0,70520.0,43828.0,60833.0,516.0
GNRC Region,2004.0,865173.0,8652.0,439.0,4729.0,53160.0,97956.0,37553.0,96070.0,34691.0,21040.0,36820.0,14964.0,41510.0,7225.0,61188.0,62132.0,95216.0,14213.0,72177.0,43886.0,61072.0,478.0
GNRC Region,2005.0,886623.0,8463.0,550.0,4723.0,55530.0,97997.0,38552.0,98949.0,35240.0,21558.0,37089.0,15330.0,43077.0,7202.0,64118.0,64691.0,97755.0,14648.0,75825.0,43523.0,61470.0,336.0


In [272]:
#first we will aggregate some of these industries to match our other industry data
data['Total'] = data['Total - All Industries']
data['Natural Resources & Mining'] = data['Agriculture, Forestry, Fishing and Hunting (11)'] + data['Mining, Quarrying, and Oil and Gas Extraction (21)']
data['Construction'] = data['Construction (23)']
data['Manufacturing'] = data['Manufacturing (31)']
thelist = [data['Wholesale Trade (42)'], data['Retail Trade (44)'], data['Transportation and Warehousing (48)'], data['Utilities (22)']]
data['Trade, Transportation, & Utilities'] = sum(thelist)
data['Information'] = data['Information (51)']
thelist = [data['Finance and Insurance (52)'], data['Real Estate and Rental and Leasing (53)']]
data['Financial Activities'] = sum(thelist)
thelist = [data['Professional, Scientific, and Technical Services (54)'], data['Management of Companies and Enterprises (55)'], 
           data['Administrative and Support and Waste Management and Remediation Services (56)']]
data['Professional & Business Services'] = sum(thelist)
thelist = [data['Educational Services (61)'], data['Health Care and Social Assistance (62)']]
data['Education & Health Services'] = sum(thelist)
thelist = [data['Arts, Entertainment, and Recreation (71)'], data['Accommodation and Food Services (72)']]
data['Leisure & Hospitality'] = sum(thelist)
data['Public Administration'] = data['Public Administration (92)']
thelist = [data['Other Services (except Public Administration) (81)'], data['Unclassified (99)']]
data['Other/Unclassified'] = sum(thelist)
thelist = [data['Natural Resources & Mining'], data['Construction'], data['Manufacturing']]
data['Goods Producing'] = sum(thelist)
thelist = [data['Information'], data['Financial Activities'], data['Professional & Business Services'], 
           data['Education & Health Services'], data['Leisure & Hospitality'], data['Public Administration'], data['Other/Unclassified']]
data['Service Producing'] = sum(thelist)

cols = ['Agriculture, Forestry, Fishing and Hunting (11)', 'Mining, Quarrying, and Oil and Gas Extraction (21)', 
        'Wholesale Trade (42)', 'Retail Trade (44)', 'Transportation and Warehousing (48)', 'Utilities (22)', 'Finance and Insurance (52)', 
        'Real Estate and Rental and Leasing (53)', 'Professional, Scientific, and Technical Services (54)', 'Management of Companies and Enterprises (55)', 
        'Administrative and Support and Waste Management and Remediation Services (56)', 'Educational Services (61)', 'Health Care and Social Assistance (62)', 
        'Arts, Entertainment, and Recreation (71)', 'Accommodation and Food Services (72)', 'Other Services (except Public Administration) (81)', 
        'Unclassified (99)', 'Total - All Industries', 'Construction (23)', 'Manufacturing (31)', 'Information (51)', 'Public Administration (92)']
data = data.drop(columns = cols)

In [274]:
data.head()

,Year,Total,Natural Resources & Mining,Construction,Manufacturing,"Trade, Transportation, & Utilities",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Public Administration,Other/Unclassified,Goods Producing,Service Producing
NAME,,,,,,,,,,,,,,,
GNRC Region,2001.0,833914.0,9698.0,52041.0,105210.0,166755.0,24400.0,52254.0,103957.0,139299.0,80486.0,57873.0,41941.0,166949.0,500210.0
GNRC Region,2002.0,828378.0,9606.0,50012.0,99314.0,161873.0,23209.0,50333.0,106379.0,144166.0,81196.0,58987.0,43303.0,158932.0,507573.0
GNRC Region,2003.0,844281.0,9169.0,52876.0,97117.0,168585.0,21799.0,52208.0,101418.0,151504.0,84428.0,60833.0,44344.0,159162.0,516534.0
GNRC Region,2004.0,865171.0,9091.0,53160.0,97956.0,173043.0,21040.0,51784.0,109923.0,157348.0,86390.0,61072.0,44364.0,160207.0,531921.0
GNRC Region,2005.0,886626.0,9013.0,55530.0,97997.0,177464.0,21558.0,52419.0,114397.0,162446.0,90473.0,61470.0,43859.0,162540.0,546622.0


In [273]:
#resum so that percentages work - data caveat that they would not sum perfectly due to rounding
thelist = [data['Natural Resources & Mining'], data['Construction'], data['Manufacturing'], data['Trade, Transportation, & Utilities'], 
           data['Information'], data['Financial Activities'], data['Professional & Business Services'], data['Education & Health Services'], 
           data['Leisure & Hospitality'], data['Public Administration'], data['Other/Unclassified']]
data['Total'] = sum(thelist)

In [275]:
cols = data.columns
thing = data.reset_index(drop = False).melt(var_name = 'Industry', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Employment')
thing.head()

,NAME,Year,Industry,Employment
0,GNRC Region,2001.0,Total,833914.0
1,GNRC Region,2002.0,Total,828378.0
2,GNRC Region,2003.0,Total,844281.0
3,GNRC Region,2004.0,Total,865171.0
4,GNRC Region,2005.0,Total,886626.0


In [276]:
percents = data

In [277]:
percents.head()

,Year,Total,Natural Resources & Mining,Construction,Manufacturing,"Trade, Transportation, & Utilities",Information,Financial Activities,Professional & Business Services,Education & Health Services,Leisure & Hospitality,Public Administration,Other/Unclassified,Goods Producing,Service Producing
NAME,,,,,,,,,,,,,,,
GNRC Region,2001.0,833914.0,9698.0,52041.0,105210.0,166755.0,24400.0,52254.0,103957.0,139299.0,80486.0,57873.0,41941.0,166949.0,500210.0
GNRC Region,2002.0,828378.0,9606.0,50012.0,99314.0,161873.0,23209.0,50333.0,106379.0,144166.0,81196.0,58987.0,43303.0,158932.0,507573.0
GNRC Region,2003.0,844281.0,9169.0,52876.0,97117.0,168585.0,21799.0,52208.0,101418.0,151504.0,84428.0,60833.0,44344.0,159162.0,516534.0
GNRC Region,2004.0,865171.0,9091.0,53160.0,97956.0,173043.0,21040.0,51784.0,109923.0,157348.0,86390.0,61072.0,44364.0,160207.0,531921.0
GNRC Region,2005.0,886626.0,9013.0,55530.0,97997.0,177464.0,21558.0,52419.0,114397.0,162446.0,90473.0,61470.0,43859.0,162540.0,546622.0


In [278]:
#first we will aggregate some of these industries to match our other industry data
percents['Natural Resources & Mining'] = percent(percents['Natural Resources & Mining'], percents['Total'])
percents['Construction'] = percent(percents['Construction'], percents['Total'])
percents['Manufacturing'] = percent(percents['Manufacturing'], percents['Total'])
percents['Trade, Transportation, & Utilities'] = percent(percents['Trade, Transportation, & Utilities'], percents['Total'])
percents['Information'] = percent(percents['Information'], percents['Total'])
percents['Financial Activities'] = percent(percents['Financial Activities'], percents['Total'])
percents['Professional & Business Services'] = percent(percents['Professional & Business Services'], percents['Total'])
percents['Education & Health Services'] = percent(percents['Education & Health Services'], percents['Total'])
percents['Leisure & Hospitality'] = percent(percents['Leisure & Hospitality'], percents['Total'])
percents['Public Administration'] = percent(percents['Public Administration'], percents['Total'])
percents['Other/Unclassified'] = percent(percents['Other/Unclassified'], percents['Total'])
percents['Goods Producing'] = percent(percents['Goods Producing'], percents['Total'])
percents['Service Producing'] = percent(percents['Service Producing'], percents['Total'])

In [279]:
thing.head()

,NAME,Year,Industry,Employment
0,GNRC Region,2001.0,Total,833914.0
1,GNRC Region,2002.0,Total,828378.0
2,GNRC Region,2003.0,Total,844281.0
3,GNRC Region,2004.0,Total,865171.0
4,GNRC Region,2005.0,Total,886626.0


In [280]:
cols = percents.columns
percents = percents.reset_index(drop = False).melt(var_name = 'Industry', id_vars = ['NAME', 'Year'], value_vars = cols, value_name = 'Percent of Employment')
percents.head()

,NAME,Year,Industry,Percent of Employment
0,GNRC Region,2001.0,Total,833914.0
1,GNRC Region,2002.0,Total,828378.0
2,GNRC Region,2003.0,Total,844281.0
3,GNRC Region,2004.0,Total,865171.0
4,GNRC Region,2005.0,Total,886626.0


In [281]:
percents = percents.set_index('Industry').transpose()
percents = percents.drop(columns = 'Total')
percents = percents.transpose().reset_index(drop = False)

In [282]:
df = thing.merge(percents, on = ['Industry', 'Year', 'NAME'], how = 'outer')
df.head(20)

,NAME,Year,Industry,Employment,Percent of Employment
0,GNRC Region,2001.0,Total,833914.0,NaN
1,GNRC Region,2002.0,Total,828378.0,NaN
2,GNRC Region,2003.0,Total,844281.0,NaN
3,GNRC Region,2004.0,Total,865171.0,NaN
4,GNRC Region,2005.0,Total,886626.0,NaN
5,GNRC Region,2006.0,Total,910451.0,NaN
6,GNRC Region,2007.0,Total,918193.0,NaN
7,GNRC Region,2008.0,Total,907869.0,NaN
8,GNRC Region,2009.0,Total,866292.0,NaN
9,GNRC Region,2010.0,Total,868743.0,NaN


In [283]:
df = df.set_index('NAME').transpose()

In [284]:
df = df.rename(columns = {'Adams city, TN': 'Adams', 'Ashland City town, TN': 'Ashland City', 'Belle Meade city, TN': 'Belle Meade',
                                    'Berry Hill city, TN': 'Berry Hill', 'Brentwood city, TN': 'Brentwood', 'Burns town, TN': 'Burns',
                                    'Cedar Hill city, TN': 'Cedar Hill', 'Charlotte town, TN': 'Charlotte',
                                    'Cheatham County, TN': 'Cheatham County', 'Clarksville city, TN': 'Clarksville',
                                    'Columbia city, TN': 'Columbia', 'Coopertown town, TN': 'Coopertown',
                                    'Cross Plains city, TN': 'Cross Plains', 'Cumberland City town, TN': 'Cumberland City',
                                    'Davidson County, TN': 'Davidson County', 'Dickson city, TN': 'Dickson',
                                    'Dickson County, TN': 'Dickson County', 'Dover city, TN': 'Dover',
                                    'Eagleville city, TN': 'Eagleville', 'Erin city, TN': 'Erin', 'Fairview city, TN': 'Fairview',
                                    'Forest Hills city, TN': 'Forest Hills', 'Franklin city, TN': 'Franklin', 'Gallatin city, TN': 'Gallatin',
                                    'Goodlettsville city, TN': 'Goodlettsville', 'Greenbrier town, TN': 'Greenbrier',
                                    'Hartsville/Trousdale County, TN': 'Hartsville/Trousdale', 'Hendersonville city, TN': 'Hendersonville',
                                    'Houston County, TN': 'Houston County', 
                                    'Kingston Springs town, TN': 'Kingston Springs', 'La Vergne city, TN': 'La Vergne',
                                    'Lebanon city, TN': 'Lebanon', 'McEwen city, TN': 'McEwen',
                                    'Millersville city, TN': 'Millersville', 'Mitchellville city, TN': 'Mitchellville',
                                    'Montgomery County, TN': 'Montgomery County', 'Mount Juliet city, TN': 'Mount Juliet',
                                    'Mount Pleasant city, TN': 'Mount Pleasant', 'Murfreesboro city, TN': 'Murfreesboro',
                                    'Nashville-Davidson metropolitan government (balance), TN': 'Nashville', 
                                    'New Johnsonville city, TN': 'New Johnsonville', 'Nolensville town, TN': 'Nolensville',
                                    'Oak Hill city, TN': 'Oak Hill', 'Pegram town, TN': 'Pegram', 'Pleasant View city, TN': 'Pleasant View',
                                    'Portland city, TN': 'Portland', 'Ridgetop city, TN': 'Ridgetop',
                                    'Rutherford County, TN': 'Rutherford County', 'Slayden town, TN': 'Slayden', 'Smyrna town, TN': 'Smyrna',
                                    'Spring Hill city, TN': 'Spring Hill', 'Springfield city, TN': 'Springfield',
                                    'Stewart County, TN': 'Stewart County', 'Sumner County, TN': 'Sumner County',
                                    'Tennessee Ridge town, TN': 'Tennessee Ridge', "Thompson's Station town, TN": "Thompson's Station",
                                    'Trousdale County, TN': 'Trousdale County', 'Vanleer town, TN': 'Vanleer', 'Watertown city, TN': 'Watertown',
                                    'Waverly city, TN': 'Waverly', 'Westmoreland town, TN': 'Westmoreland', 'White Bluff town, TN': 'White Bluff',
                                    'White House city, TN': 'White House', 'Williamson County, TN': 'Williamson County',
                                    'Wilson County, TN': 'Wilson County'})
df = df.transpose()
df = df.reset_index()

In [285]:
df.tail(200)

,NAME,Year,Industry,Employment,Percent of Employment
20730,Tennessee Ridge,2015.0,Service Producing,83.0,32.421875
20731,Tennessee Ridge,2016.0,Service Producing,86.0,38.738739
20732,Tennessee Ridge,2017.0,Service Producing,108.0,49.769585
20733,Tennessee Ridge,2018.0,Service Producing,121.0,48.594378
20734,Tennessee Ridge,2019.0,Service Producing,114.0,48.101266
20735,Tennessee Ridge,2020.0,Service Producing,115.0,51.339286
20736,Tennessee Ridge,2021.0,Service Producing,119.0,52.422907
20737,Thompson's Station,2001.0,Service Producing,346.0,77.927928
20738,Thompson's Station,2002.0,Service Producing,355.0,78.021978
20739,Thompson's Station,2003.0,Service Producing,339.0,80.331754


In [286]:
df.to_csv('../../Outputs/JOBSEQ_INDUSTRYEMP__PLACES_HISTORICALANDCURRENT.csv', index = False)